# Event Analysis
This jupyter notebook will show you how to use the Event Analysis Library You can install the library by running
```
pip install event-analysis
```
If you have an Nvidia GPU and you want to accelerate your workload using the GPU, you will need to install PyCuda which you can by running
```
pip install pycuda
```
The library provides functions for Event Synchronisation and Event Coincidence Analysis. Read the documentation if you want to learn more about them.



In [2]:
!pip install geopandas pycuda event-analysis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 15.2 MB/s 
     |████████████████████████████████| 1.7 MB 48.7 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 16.7 MB 824 kB/s 
     |████████████████████████████████| 6.3 MB 48.9 MB/s 
     |████████████████████████████████| 78 kB 8.5 MB/s 
     |████████████████████████████████| 70 kB 9.3 MB/s 
  Created wheel for pycuda: filename=pycuda-2022.1-cp37-cp37m-linux_x86_64.whl size=629701 sha256=982df9bd2e81cf6224acdddcab20610eeda230917fe421832855b86e06607979
  Stored in directory: /root/.cache/pip/wheels/17/53/c9/caa05618e686df51f017d8a9923f38d915ce31df67ab6628e6
  Created wheel for pytools: filename=pytools-2022.1.12-py2.py3-none-any.whl size=65033 sha256=530065f90c18c704e14ad1ab8b419026d0408cffe03e22cadfcc82e443ddeea3
 

To use the library you first need to create a pandas dataframe. The dataframe must satisfy 2 properties

1) The index must be an increasing time series of python datetime objects*
2) The internal data of the dataframe must be boolean

The columns will be the name of the event series

*1) Internally the library will store the timeseries in an int32 array such that

    timeseries[i] = timeseries[0] + deltatime(hour = array[i])
This means if your timeseries has time difference's which are not multiple of an hour then the algorithm will give wrong results. If your time series spans over 65 years i.e

    (timeseries[-1] - timeseries[0]).years > 65
then an integer overflow will cause the algorithm to give wrong results. If your specific use case meets either of these conditions please open an issue, or better yet fix them yourself and open a pull request! Contributions are much appritiated.

Inside the library I have written helper function to convert my data into this format. You will have to write your own.



In [19]:
import datetime
import numpy as np
import pandas as pd
import geopandas as gpd
from helperfunction import numpy_from_csv_data, get_Df_From_numpy

In [5]:
india_mp_gdf = gpd.read_file("/content/drive/MyDrive/shapefile/Ind.shp")
map_shape_object = india_mp_gdf.geometry
file_name = "/content/drive/MyDrive/trmm_3hrs_precp_2003_2019.csv"

In [6]:
numpy_mat, missing_vals_index_set, viable_columns = numpy_from_csv_data(file_name, map_shape_object)
starting_date = datetime.datetime(2003,1,1,hour=0,minute=0,second=0,microsecond=0)
time_delta = datetime.timedelta(hours=3)
rainDataDf = get_Df_From_numpy(numpy_mat,  viable_columns, starting_date = starting_date, time_delta = time_delta)
monsoonDataDf = rainDataDf[rainDataDf.index.month >= 6 | (rainDataDf.index.month <=9) ]
monsoonEventDf = monsoonDataDf > np.percentile(monsoonDataDf.to_numpy(copy = True), 98, interpolation = "lower", overwrite_input = True)


Dropping Not Point elements / Points outside shape
Number of elements dropped :: 45002
Starting main Loop :: Remove Missing Values


14384it [00:52, 273.21it/s]


Number of values dropped :: 115526


I am going to be analysing the rainfall events over the Indian sub-continent at 3 hour different from the year 2003 to 2019

In [20]:
monsoonEventDf.head()

Coordinates,X93.875Y6.875,X93.625Y7.375,X77.375Y8.125,X77.125Y8.375,X77.375Y8.375,X77.625Y8.375,X77.875Y8.375,X76.875Y8.625,X77.125Y8.625,X77.375Y8.625,...,X74.875Y36.625,X75.125Y36.625,X75.375Y36.625,X75.625Y36.625,X73.875Y36.875,X74.375Y36.875,X74.625Y36.875,X74.875Y36.875,X75.125Y36.875,X75.375Y36.875
DateTime,,,,,,,,,,,,,,,,,,,,,
2003-07-01 00:00:00,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2003-07-01 03:00:00,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2003-07-01 06:00:00,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2003-07-01 09:00:00,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
2003-07-01 12:00:00,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [9]:
from EventAnalysis import EventAnalysis

In [10]:
EA = EventAnalysis(monsoonEventDf)

In [12]:
ES_Q = EA.ES_Cuda()

Device ID was not specified, reverting to 0
Device in use 
	Tesla T4
Time elapsed to run the computation :: 2 minutes 24 seconds 


In [14]:
ES_Q['X68.125Y23.625']['X77.375Y8.125']

3.0

In [16]:
EC_p_max,EC_p_mean, EC_t_max,EC_t_mean, p_prec, p_trig = EA.ECA_Cuda(datetime.timedelta(hours = 100), return_p_values = True)

Time elapsed to run the computation :: 0 minutes 5 seconds 


In [17]:
EC_t_max['X68.125Y23.625']['X77.375Y8.125']

0.088

In [18]:
for ECA_res in EA.ECA_vec_Cuda([datetime.timedelta(hours = h) for h in range(3, 30, 10)]):
  EC_p_max,EC_p_mean, EC_t_max,EC_t_mean = ECA_res

Time elapsed to run the computation :: 0 minutes 4 seconds 
